In [1]:
# hyper parameter tuning
# using FCN, loss = mse

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, glob
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # GPU
import tensorflow.python as tf

import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
from scipy import signal, ndimage
from scipy.io import loadmat, savemat
from skimage import data, img_as_float, exposure
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

In [3]:
from func_processing import img_affine, img_hflip, load, get_y_as_heatmap, get_avg_xy
from sklearn.model_selection import train_test_split

FTRAIN = 'facial-keypoints-detection/training/training.csv'

In [4]:
# load data
X, Y = load(FTRAIN)
X = X.reshape(-1,96,96)
# split into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
X_train = np.expand_dims(X_train, axis=3)
X_test = np.expand_dims(X_test, axis=3)

In [5]:
# training data augmentation
np.random.seed(0)
dz = np.random.randint(-5,5,(X_train.shape[0],2))
X_train0 = []
Y_train0 = []
for i, x in enumerate(X_train):
    xt, yt = img_affine(np.squeeze(x), Y_train[i], dz[i,0], dz[i,1], 96, 96)
    X_train0.append(xt)
    Y_train0.append(yt)
    xf, yf = img_hflip(np.squeeze(x), Y_train[i])
    X_train0.append(xf)
    Y_train0.append(yf)
X_train0 = np.array(X_train0)
Y_train0 = np.array(Y_train0)

In [6]:
X_train0 = np.expand_dims(X_train0, axis=3)

In [7]:
from func_FCN import FCN
SIGMA = [5, 7, 9, 11]
N = [2, 3, 4, 5]
ERR = np.zeros((4, 4))
MIN_ERR = np.inf
for p,sigma in enumerate(SIGMA):
    for q,n in enumerate(N):
        y_train0 = get_y_as_heatmap(Y_train0, 96, 96, sigma)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train0, y_train0, test_size=0.2, random_state=43)

        model = FCN(96, 96, n*32)
        model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

        from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
        callbacks = [
        EarlyStopping(patience=10, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
        ModelCheckpoint('model-tgs-seg2.h5', verbose=1, save_best_only=True, save_weights_only=True)
        ]

        history = model.fit(X_train, y_train, batch_size=32, epochs=50, callbacks=callbacks, validation_data=(X_valid, y_valid))

        plt.plot(history.history['loss'], label='traning loss')
        plt.plot(history.history['val_loss'], label = 'validation loss')
        plt.yscale('log')
        plt.legend()
        plt.xlabel('Epoch')
        plt.ylabel('loss')
        plt.savefig('./model/hp/hist_sigma%d_n%d.png'%(sigma, n))
        plt.close()

        model_predict = model.predict(X_test)
        y_pred = np.zeros((model_predict.shape[0], 30))
        for i in range(model_predict.shape[0]):
            for j in range(15):
                msk = np.squeeze(model_predict[i,:,:,j])
                yx, prob = get_avg_xy(msk, 25)
                y_pred[i,2*j] = yx[1] / 96
                y_pred[i,2*j+1] = yx[0] / 96
        error = np.linalg.norm(Y_test - y_pred)
        ERR[p,q] = error
        np.savetxt('fcn_hp.txt',(sigma,n,error),fmt='.1f',delimiter=',')
        if error < MIN_ERR:
            MIN_ERR = error
            MIN_SIGMA = sigma
            MIN_N = n

mprove from 0.00437
3081/3081 [==============================] - 8s 2ms/sample - loss: 6.4833e-04 - acc: 0.8840 - val_loss: 0.0044 - val_acc: 0.7989
Epoch 26/50
3072/3081 [============================>.] - ETA: 0s - loss: 6.1083e-04 - acc: 0.8863
Epoch 00026: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 00026: val_loss did not improve from 0.00437
3081/3081 [==============================] - 8s 3ms/sample - loss: 6.1121e-04 - acc: 0.8863 - val_loss: 0.0044 - val_acc: 0.8033
Epoch 27/50
3072/3081 [============================>.] - ETA: 0s - loss: 5.3993e-04 - acc: 0.8911
Epoch 00027: val_loss improved from 0.00437 to 0.00432, saving model to model-tgs-seg2.h5
3081/3081 [==============================] - 8s 2ms/sample - loss: 5.4006e-04 - acc: 0.8911 - val_loss: 0.0043 - val_acc: 0.8041
Epoch 28/50
3072/3081 [============================>.] - ETA: 0s - loss: 5.2780e-04 - acc: 0.8917
Epoch 00028: val_loss did not improve from 0.00432
3081/3081 [==============

In [ ]:
# sigma = 5, N = 32 * 2